## README

- spikes are given as input instead of rr interval values

### Time taken to run ANN(approx)

| Length of input array | Time taken to run |
| --------------------- | ----------------- |
| 10_000_000            | 19 min            |
| 1_000_000             | 1 min 15 sec      |
| 100_000               | 10 sec            |
| 3600                  | 0.2 sec           |

### Time taken to run SNN(approx, excluding ANN)

| Length of input array | Time taken to run |
| --------------------- | ----------------- |
| 10_000_000            | todo              |
| 1_000_000             | 30 min            |
| 100_000               | 3 min 30 sec      |
| 3600                  | 6 sec             |


## Importing Packages

In [1]:
import random
import numpy as np
random.seed(123)

In [2]:
steps = 64

## Declaring new class IF_Neuron

In [3]:
# Integrate and Fire Neuron Object


class IF_Neuron:
    def __init__(self, layer, weights, bias, steps, v_th):
        self.layer = layer  # Layer of Neuron
        self.weights = weights  # Weights
        self.bias = bias  # Bias

        # IF Properties
        self.steps = steps  # Number of steps for calculation
        self.v = np.empty([self.steps])  # Neuron Value
        self.v[0] = 0  # Set start Value
        self.spikes = np.empty([self.steps])  # Output spike train
        self.n_spikes = 0  # Number of spikes
        self.v_th = v_th  # Threshold

    def reset(self):  # Reset Neuron
        self.v = np.empty([self.steps])
        self.v[0] = 0
        self.spikes = np.empty([self.steps])
        self.n_spikes = 0

    def calculate(self, neuron_input):
        for i in range(self.steps):  # for every step
            if i > 0:  # not first element
                self.v[i] = self.v[i - 1]
            for j in range(len(neuron_input)):  # for every input channel
                if neuron_input[j][i]:  # if spike present
                    self.v[i] = self.v[i] + self.weights[j]
            self.v[i] = self.v[i] + self.bias
            if self.v[i] > self.v_th:  # if above threshold
                self.n_spikes += 1  # increase number of spikes
                self.spikes[i] = 1  # add spike to spike trace
                self.v[i] = self.v[i] - self.v_th  # difference reset
                # self.Vm[i] = self.bias            # hard reset
            else:  # add no spike to spike trace
                self.spikes[i] = 0

## Neuron model for ANN

In [4]:
# Neuron Object for ANN
class Neuron:
    def __init__(self, layer, weights, bias):
        self.layer = layer  # layer
        self.weights = weights  # weights
        self.bias = bias  # bias
        self.output = 0  # output

    def calculate(self, inputs):  # calculation
        relu_activation = ReLU()
        x = np.dot(self.weights, inputs) + self.bias
        self.output = relu_activation.calculate(x)


# ReLU activation function
class ReLU:
    def __init__(self):
        self.output = 0

    def calculate(self, x):
        self.output = np.maximum(0, x)  # relu function
        return self.output

## `load_model`

In [5]:
# network structure, weights, biases and activation of output neurons of existing ANN
def load_model():
    network_structure = [2, 8, 7]  # input layer, hidden layer, output layer
    weights = np.array(
        [
            [3.8722e00, -1.9211e-01],
            [5.3290e00, 4.7047e-01],
            [-5.1274e00, -5.1903e-03],
            [-2.1598e-01, 3.8066e00],
            [8.6600e-02, -8.0032e00],
            [5.6373e00, -1.5221e-01],
            [1.2375e00, 4.9385e-01],
            [3.5665e00, 3.5465e-01],
            [1.4272, -6.1945, -2.0182, 1.3436, 2.2910, 0.4987, 0.3354, -3.3586],
            [-1.1558, -0.3980, -3.7244, -1.4081, -2.3210, -1.6481, -2.1048, -0.4278],
            [-8.2870, -1.0868, 0.8291, 0.7862, -21.4690, -17.1373, -0.5934, -0.1413],
            [-4.1481, -0.5656, 2.4139, -9.3905, -2.0829, -16.9980, -0.0992, -0.9269],
            [0.5731, 1.5926, -18.0025, -0.3991, -16.3425, 0.7215, 0.3035, 1.3948],
            [0.8768, 0.5940, -13.4116, -7.6260, 0.3059, 1.4851, 0.7284, 0.3347],
            [-7.8029, -0.9323, 0.8822, -36.0286, 3.4898, -18.0212, -3.9245, -0.6883],
        ],
        dtype=object,
    )
    biases = np.array(
        [
            3.1150,
            -1.4917,
            2.9950,
            2.6293,
            -2.0631,
            3.2291,
            1.8875,
            -1.5298,
            0.7810,
            -1.4082,
            -1.0581,
            1.9883,
            -1.8215,
            0.3030,
            -1.5795,
        ],
    )
    classification_threshold = 0.5  # output activation needed to assign to class

    return network_structure, weights, biases, classification_threshold

## `create_neurons`

In [6]:
def create_neurons(structure, weights, biases, steps, v_th, net_type):
    neuron_array = []  # array in which the neuron objects will be saved
    neuron_number = 0  # counter to initialize/create the neuron objects

    for layer in range(
        1, len(structure)
    ):  # for every layer (except layer 0 which is input)
        for neuron in range(structure[layer]):  # for number of neurons in layer
            if net_type == "SNN":  # create SNN neurons
                neuron_array.append(
                    IF_Neuron(
                        layer,
                        weights[neuron_number],
                        biases[neuron_number],
                        steps,
                        v_th[layer - 1],
                    )
                )
            else:  # create ANN neurons
                neuron_array.append(
                    Neuron(layer, weights[neuron_number], biases[neuron_number])
                )
            neuron_number += 1

    return neuron_array  # array of neuron objects

## `reset_neurons`

In [7]:
def reset_neurons(neuron_array):
    for neuron in neuron_array:
        neuron.reset()

## `create_input_spike`

In [8]:
def create_input_spike(input_array, steps):
    input_spike = np.zeros(steps)
    # rr_values_array=rr_values_array.astype(int)
    # for i, value in enumerate(rr_values_array):
    for l in range(steps):
        rnd = random.randrange(60, 120)
        # print(f"rnd:{rnd}")
        # print(f"input_array:{rr_values_array}")
        print(f"input_spike:{input_spike}")
        if rnd < input_array:
            input_spike[l] = 1

    return input_spike

## `create_input_spikes`

In [9]:
def create_input_spikes(input_array, steps):
    input_spikes = np.zeros(shape=(2, steps))
    for i, interval in enumerate(input_array):  
        for step in range(steps):  
            rnd = random.randrange(0, 2)
            if rnd < interval:  # create spikes
                input_spikes[i][step] = 1

    return input_spikes

## `calculate_pixel`

In [10]:
def calculate_pixel(network_structure, neuron_array, input_elements, net_type):
    max_activation = [0] * len(network_structure)
    for layer in range(1, len(network_structure)):  # for layer
        if layer == 1:  # if input layer
            neuron_input_array = input_elements  # get img values as input
        else:  # otherwise, get outputs of previous layer
            neuron_input_array = []
            for neuron in neuron_array:
                if neuron.layer == layer - 1:
                    if net_type == "SNN":
                        neuron_input_array.append(neuron.spikes)  # spikes output
                    else:
                        neuron_input_array.append(neuron.output)
        for neuron in neuron_array:  # get all neurons of current layer and calculate
            if neuron.layer == layer:
                if net_type == "SNN":
                    neuron.calculate(neuron_input_array)
                else:
                    neuron.calculate(neuron_input_array)
                    max_activation[layer] = max(max_activation[layer], neuron.output)
    return max_activation  # max activation per layer

## `calculate_network`

In [11]:
def calculate_network(network_structure, neuron_array, input_array, steps, net_type):
    output_array = []
    max_activation = [0] * len(network_structure)
    max_activation[0] = 1
    number_of_total_spikes = 0

    for i, pixel in enumerate(input_array):  # for every pixel
        if (i + 1) / len(input_array) * 100 % 1 == 0:  # print status update
            print("\r", f"{int((i+1)/len(input_array)*100)}% done", end="")
        if net_type == "SNN":  # if SNN
            reset_neurons(neuron_array)  # reset neurons
            input_spikes = create_input_spikes(pixel, steps)  # convert input to spikes
            calculate_pixel(network_structure, neuron_array, input_spikes, net_type)
        else:  # if ANN
            temp_max_act = calculate_pixel(
                network_structure, neuron_array, pixel, net_type
            )
            for j, element in enumerate(temp_max_act):
                max_activation[j] = max(max_activation[j], element)
        temp_out = []  # temporary array to store outputs
        for neuron in neuron_array:  # for every neuron
            if net_type == "SNN":
                number_of_total_spikes += neuron.n_spikes
            if neuron.layer == (len(network_structure) - 1):  # if output neuron
                if net_type == "SNN":
                    temp_out.append(
                        neuron.n_spikes
                    )  # append number of spikes per output neuron
                else:
                    temp_out.append(neuron.output)  # append value of output neuron
        output_array.append(temp_out)  # append to output values
    print("\r", f"", end="")
    output_array = np.array(output_array)  # convert tu numpy array

    return output_array, max_activation  # output values, max_activation per layer

## `normalization`

In [12]:
def data_normalization(max_activation):
    weight_factor = [1, 1]
    bias_factor = [1, 1]
    v_th = [1, 1]

    for i, value in enumerate(weight_factor):  # layer-wise adaption of weights
        weight_factor[i] = value * max_activation[i] / max_activation[i + 1]
    for i, value in enumerate(bias_factor):  # layer-wise adaption of biases
        bias_factor[i] = value / max_activation[i + 1]

    return weight_factor, bias_factor, v_th

## SNN

### `scale_model`

In [13]:
# scale the weights and biases
def scale_model(structure, weights, biases, w_factor, b_factor, fp_convert):
    scaled_weights = []
    scaled_biases = []
    factor_counter = 0
    inlayer_counter = 0

    for weight in weights:
        if fp_convert:
            scaled_weights.append([round(i * w_factor[factor_counter]) for i in weight])
        else:
            scaled_weights.append([i * w_factor[factor_counter] for i in weight])
        inlayer_counter += 1
        if inlayer_counter >= structure[factor_counter + 1]:
            inlayer_counter = 0
            factor_counter += 1
    factor_counter = 0
    inlayer_counter = 0
    for bias in biases:
        if fp_convert:
            scaled_biases.append(round(bias * b_factor[factor_counter]))
        else:
            scaled_biases.append(bias * b_factor[factor_counter])
        inlayer_counter += 1
        if inlayer_counter >= structure[factor_counter + 1]:
            inlayer_counter = 0
            factor_counter += 1
    # print(f"Scaled Weights: {scaled_weights} Bias: {scaled_biases}")  # debug

    return scaled_weights, scaled_biases

### `decode_output_spikes`

In [14]:
# get spike rate
def decode_output_spikes(spikes, steps):
    spike_ratio = (
        np.array(spikes) / steps
    )  # spike_ratio = number_of_spikes/number_of_steps

    return spike_ratio

## `create_input_array`

In [15]:
def create_input_array(length):
    input_array = np.random.randint(0, 2, size=(length, 2))
    return input_array

In [16]:
input_array = create_input_array(1_000_000)
# input_array.shape

## End program

In [17]:
network_structure, weights, biases, classification_threshold = load_model()

In [18]:
neuron_array = create_neurons(network_structure, weights, biases, 0, 0 , "ANN")

In [19]:
output_array, max_activation = calculate_network(
    network_structure, neuron_array, input_array, 0, "ANN"
)

 100% done

In [20]:
weight_factor, bias_factor, v_th = data_normalization(max_activation)
# weight_factor, bias_factor, v_th

In [21]:
snn_weights, snn_biases = scale_model(
    network_structure, weights, biases, weight_factor, bias_factor, 0
)
# snn_weights, snn_biases

In [22]:
snn_neuron_array = create_neurons(
    network_structure, snn_weights, snn_biases, steps, [1, 1], "SNN"
)

In [23]:
print("Calculating SNN...")
output_spikes, not_used = calculate_network(
    network_structure, snn_neuron_array, input_array, steps, "SNN"
)

Calculating SNN...
 100% done

In [24]:
spike_ratio = decode_output_spikes(output_spikes, steps)
spike_ratio

array([[0.390625, 0.      , 0.      , ..., 0.140625, 0.03125 , 0.      ],
       [0.265625, 0.      , 0.      , ..., 0.      , 0.      , 0.      ],
       [0.4375  , 0.      , 0.      , ..., 0.      , 0.      , 0.      ],
       ...,
       [0.265625, 0.      , 0.      , ..., 0.      , 0.      , 0.      ],
       [0.5     , 0.      , 0.      , ..., 0.03125 , 0.      , 0.      ],
       [0.359375, 0.      , 0.      , ..., 0.234375, 0.046875, 0.      ]])